In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('datasets/final_book_dataset_kaggle 2.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          946 non-null    object 
 1   author         824 non-null    object 
 2   price          944 non-null    float64
 3   pages          934 non-null    float64
 4   avg_reviews    823 non-null    float64
 5   n_reviews      946 non-null    int64  
 6   star5          946 non-null    float64
 7   star4          946 non-null    float64
 8   star3          946 non-null    float64
 9   star2          946 non-null    float64
 10  star1          946 non-null    float64
 11  dimensions     914 non-null    object 
 12  weight         943 non-null    object 
 13  language       943 non-null    object 
 14  publisher      942 non-null    object 
 15  ISBN_13        945 non-null    object 
 16  link           946 non-null    object 
 17  complete_link  946 non-null    object 
dtypes: float64

---

#### **Pré-tratamento dos valores núlos relevantes para análise,**  
- como nosso objetivo é listar os mais bem avaliados para criar uma lista de possíveis recomendações de leitura,  vamos tratar somente os valores nulos em 'author' e 'avg_reviews'

In [4]:
null = df['author'].isna()
null.sum() # identificamos 122 valores núlos na varíável autores

122

In [5]:
null = df['avg_reviews'].isna()
null.sum() # identificamos 123 valores nulos em avg_reviews

123

---

#### **O primeiro método consiste em usar o código 'ISBN_13' em conjunto com a api do google books para atualizar os valores nulos no df**
- será criado um novo dataframe 'arquivo_atualizado.csv'

In [6]:
import requests

In [7]:
# Função para buscar o nome do autor com base no ISBN
def buscar_autor(isbn):
  isbn_limpo = isbn.replace('-', '')
  url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn_limpo}"
  response = requests.get(url).json()
  try:
    # Tenta obter o nome do primeiro autor do livro encontrado
    autor = response['items'][0]['volumeInfo']['authors'][0]
    autores_str = ', '.join(autor) if isinstance(autor, list) else autor
    return autores_str
  except (KeyError, IndexError):
        # Retorna None se não encontrar um autor
    return None

# Preenche os autores faltantes
for index, row in df[df['author'].isnull()].iterrows():
    autor = buscar_autor(row['ISBN_13'])
    if autor:
        df.at[index, 'author'] = f'[{autor}]'

# Salva o arquivo CSV atualizado
df.to_csv('arquivo_atualizado.csv', index=False)

In [8]:
df_2 = pd.read_csv('datasets/arquivo_atualizado.csv')

In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          946 non-null    object 
 1   author         941 non-null    object 
 2   price          944 non-null    float64
 3   pages          934 non-null    float64
 4   avg_reviews    823 non-null    float64
 5   n_reviews      946 non-null    int64  
 6   star5          946 non-null    float64
 7   star4          946 non-null    float64
 8   star3          946 non-null    float64
 9   star2          946 non-null    float64
 10  star1          946 non-null    float64
 11  dimensions     914 non-null    object 
 12  weight         943 non-null    object 
 13  language       943 non-null    object 
 14  publisher      942 non-null    object 
 15  ISBN_13        945 non-null    object 
 16  link           946 non-null    object 
 17  complete_link  946 non-null    object 
dtypes: float64

In [10]:
null = df_2['author'].isna()
null.sum() 

5

- Restaram 5 valores nulos, dos quais poderiamos simplesmente excluir ou buscar manualmente cada autor
- Porém vamos criar um processo simples de automação com selenium para buscar e copiar os dados dos autores diretamente da amazon.

---

In [11]:
df_3 = df_2[['title', 'author', 'ISBN_13' ]]
nulls = df_3[null]
nulls

,title,author,ISBN_13
613,Python Machine Learning Workbook for Beginners...,NaN,978-1734790177
657,Data Mining: Practical Machine Learning Tools ...,NaN,978-0128042915
679,The Elements of Statistical Learning: Data Min...,NaN,978-0387848570
752,Python Cookbook Third Edition,NaN,978-1449340377
802,Python for Data Analysis: A Beginners Guide to...,NaN,979-8463514271


In [15]:
lista = np.array(nulls['ISBN_13']) # numpy list com os elementos para a pesquisa
lista

array(['978-1734790177', '978-0128042915', '978-0387848570',
       '978-1449340377', '979-8463514271'], dtype=object)

In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [17]:
# função para buscar os autores na amazon
def pesquisar_livro(valor):
    service = Service(ChromeDriverManager().install()) # instala chrome driver
    driver = webdriver.Chrome(service = service) # define webdriver como driver e abre o navegador

    driver.get("https://www.amazon.com")  # entra na amazon
    driver.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]').send_keys(valor) # encontra a barra de pesquisa e escreve a variável 'valor'
    driver.find_element(By.XPATH, '//*[@id="nav-search-submit-button"]').click()            # clicka em avançar na pesquisa
    driver.find_element(By.XPATH, '//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[2]/div/div/span/div/div/div/div[2]/div/div/div[1]/h2/a/span').click()
    # entra na primeira opção da lista
    info_autor = driver.find_element(By.XPATH, '//*[@id="bylineInfo"]').text # extrai o texto do caminho onde está descrito o autor
    title = driver.find_element(By.XPATH, '//*[@id="productTitle"]').text  # extrai o titúlo da obra
    return title, info_autor, valor # retorna os valores para as variáveis 
    driver.quit() # fecha o navegador

In [18]:
# vamos criar um novo dataframe para simplificar a formatação da coluna

df_resultados = pd.DataFrame(columns=['title', 'author', 'ISBN_13']) 

for pesquisa in lista: # loop simples para executar a pesquisa para cada item na lista e armazenar em df_resultados
    title, info_autor, valor = pesquisar_livro(pesquisa) # chamando a função
    
    df_resultados = pd.concat([df_resultados, pd.DataFrame({'title': [title], 'author': [info_autor], 'ISBN_13': [valor]})], ignore_index=True)

In [19]:
df_resultados

,title,author,ISBN_13
0,Python Machine Learning Workbook for Beginners...,by AI Publishing (Author),978-1734790177
1,Data Mining: Practical Machine Learning Tools ...,"by Ian H. Witten (Author), Eibe Frank (Author)...",978-0128042915
2,The Elements of Statistical Learning: Data Min...,"by Trevor Hastie (Author), Robert Tibshirani (...",978-0387848570
3,"Python Cookbook, Third Edition","by David Beazley (Author), Brian Jones (Author)",978-1449340377
4,Python for Data Analysis: A Beginners Guide to...,by Brady Ellison (Author) Format: Kindle Edition,979-8463514271


- alterando as strings para se adequar ao resto do dataframe

In [20]:
df_resultados['author'] = df_resultados['author'].str.replace('by', '')
df_resultados['author'] = df_resultados['author'].str.replace('(author)', '')

In [21]:
def adicionar_colchetes(texto):
    return f"[{texto}]"
df_resultados['author']  = df_resultados['author'].apply(adicionar_colchetes)

In [22]:
df_resultados

,title,author,ISBN_13
0,Python Machine Learning Workbook for Beginners...,[ AI Publishing (Author)],978-1734790177
1,Data Mining: Practical Machine Learning Tools ...,"[ Ian H. Witten (Author), Eibe Frank (Author),...",978-0128042915
2,The Elements of Statistical Learning: Data Min...,"[ Trevor Hastie (Author), Robert Tibshirani (A...",978-0387848570
3,"Python Cookbook, Third Edition","[ David Beazley (Author), Brian Jones (Author)]",978-1449340377
4,Python for Data Analysis: A Beginners Guide to...,[ Brady Ellison (Author) Format: Kindle Edition],979-8463514271


---

In [23]:
# adicionando os valores correspondentes no df
values = {
    '978-1734790177': '[ AI Publishing ]',
    '978-0128042915': '[ Ian H. Witten , Eibe Frank , Mark A. Hall , & 1 more]',
    '978-0387848570': '[ Trevor Hastie , Robert Tibshirani , Jerome Friedman ]',
    '978-1449340377': '[ David Beazley , Brian Jones ]',
    '979-8463514271': '[ Brady Ellison  Format: Kindle Edition]'
}

In [24]:
df_2['author'].fillna(value=df_2['ISBN_13'].map(values), inplace=True)

In [25]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          946 non-null    object 
 1   author         946 non-null    object 
 2   price          944 non-null    float64
 3   pages          934 non-null    float64
 4   avg_reviews    823 non-null    float64
 5   n_reviews      946 non-null    int64  
 6   star5          946 non-null    float64
 7   star4          946 non-null    float64
 8   star3          946 non-null    float64
 9   star2          946 non-null    float64
 10  star1          946 non-null    float64
 11  dimensions     914 non-null    object 
 12  weight         943 non-null    object 
 13  language       943 non-null    object 
 14  publisher      942 non-null    object 
 15  ISBN_13        945 non-null    object 
 16  link           946 non-null    object 
 17  complete_link  946 non-null    object 
dtypes: float64

In [26]:
null = df_2['author'].isna()
null.sum() 

0

#### Autores devidamente definidos, 
- agora vamos verificar os valores 'avg_reviews' nulos

In [27]:
null = df['avg_reviews'].isna() 
nulls = df[null]
nulls_2 = nulls[['avg_reviews', 'n_reviews',
       'star5', 'star4', 'star3', 'star2', 'star1']]
nulls_2

,avg_reviews,n_reviews,star5,star4,star3,star2,star1
8,NaN,0,0.0,0.0,0.0,0.0,0.0
23,NaN,0,0.0,0.0,0.0,0.0,0.0
37,NaN,0,0.0,0.0,0.0,0.0,0.0
47,NaN,0,0.0,0.0,0.0,0.0,0.0
51,NaN,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
842,NaN,0,0.0,0.0,0.0,0.0,0.0
845,NaN,0,0.0,0.0,0.0,0.0,0.0
928,NaN,0,0.0,0.0,0.0,0.0,0.0
929,NaN,0,0.0,0.0,0.0,0.0,0.0


In [28]:
nulls_2.var()

avg_reviews    NaN
n_reviews      0.0
star5          0.0
star4          0.0
star3          0.0
star2          0.0
star1          0.0
dtype: float64

#### Os dois ultimos métodos nos permite perceber que todos os valores nulos em 'n_reviews' são referênte a livros com nenhuma avaliação, 
- vou alterar os valores nulos da coluna por 0, considerando que não irá causar impacto nas somas de média.

In [32]:
df_2['avg_reviews'].fillna(0, inplace = True)

In [33]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          946 non-null    object 
 1   author         946 non-null    object 
 2   price          944 non-null    float64
 3   pages          934 non-null    float64
 4   avg_reviews    946 non-null    float64
 5   n_reviews      946 non-null    int64  
 6   star5          946 non-null    float64
 7   star4          946 non-null    float64
 8   star3          946 non-null    float64
 9   star2          946 non-null    float64
 10  star1          946 non-null    float64
 11  dimensions     914 non-null    object 
 12  weight         943 non-null    object 
 13  language       943 non-null    object 
 14  publisher      942 non-null    object 
 15  ISBN_13        945 non-null    object 
 16  link           946 non-null    object 
 17  complete_link  946 non-null    object 
dtypes: float64

---

#### Agora vamos criar um novo dataset já com os devidos valores tratados

In [34]:
df_2.to_csv('datasets/data_set_completo_1.csv', index = False)